## The mutation being delployed looks like this
```
apiVersion: apps.dominodatalab.com/v1alpha1
kind: Mutation
metadata:
  name: add-dataset-model
  namespace: domino-platform
rules:
  - insertVolumeMounts:
      containerSelector:
      - model
      volumeMounts:
      - mountPath: <DATASET_PATH>
        name: prediction-data-volume
        subPath: filecache/<RESOURCE_ID>
        readOnly: true
    jqSelector:
      query: |
        include "domsed/selectors/common";
        $__kind__ == "Pod" and
        (.metadata.labels."dominodatalab.com/workload-type" | isIn(["ModelAPI"])) and
        (.metadata.labels."dominodatalab.com/project-id" | isIn(["<MODEL_PROJECT_ID>"]))
```

There are three variables
1. MODEL_PROJECT_ID
2. RESOURCE_ID
3. DATASET_PATH

Get the following using the function below

In [ ]:
## Fetch Datasets for a owner/project/name
import os
import requests
def get_dataset(owner,project_id,ds_name):
    api_proxy = os.environ["DOMINO_API_PROXY"]
    endpoint = "/v4/datasetrw/datasets-v2"
    url = f"{api_proxy}{endpoint}"
    out = requests.get(url)
    lst = out.json()
    for l in lst:
        owners = l['datasetRwDto']['ownerUsernames']
        readWriteSnapshotId = l['datasetRwDto']['readWriteSnapshotId']
        p_id = l['datasetRwDto']['projectId']
        name = l['datasetRwDto']['name']
        dataset_path = l['datasetRwDto']['datasetPath']
        if owner in owners and p_id==project_id and name==ds_name:
            endpoint= f"/v4/datasetrw/snapshot/{readWriteSnapshotId}"
            url = f"{api_proxy}{endpoint}"
            out = requests.get(url)
            resource_id = out.json()['snapshot']['resourceId']
            sub_path = f"filecache/{resource_id}"
            return {"MODEL_PROJECT_ID":project_id,"RESOURCE_ID":resource_id,"DATASET_PATH":dataset_path}
    return None
    

In [ ]:
project_id = os.environ['DOMINO_PROJECT_ID']
owner = "integration-test"
ds_name = "llmstore"
print(get_dataset(owner,project_id,ds_name))